# 🔍 RAG with ChromaDB - Question & Answer System

This notebook implements a **RAG (Retrieval-Augmented Generation)** system using:
- **ChromaDB** as vector database
- **Sentence Transformers** for embeddings (FREE)
- **LangChain** for orchestration
- **TinyLlama** - 100% FREE & UNLIMITED local LLM (runs on Colab GPU)

> ⚡ **No API key needed!** Everything runs locally on Colab's free GPU.


## 1️⃣ Install Dependencies

In [20]:
# Install libraries - 100% FREE & UNLIMITED
# The model runs locally on Colab's GPU (no API key needed)
!pip install -q --upgrade langchain langchain-core langchain-community
!pip install -q langchain-text-splitters
!pip install -q chromadb sentence-transformers
!pip install -q pypdf python-docx unstructured
!pip install -q transformers accelerate bitsandbytes  # For local model
!pip install -q tiktoken

print("✅ Installation completed")
print("⚠️ IMPORTANT: Make sure GPU is enabled:")
print("   Runtime > Change runtime type > T4 GPU")

✅ Installation completed
⚠️ IMPORTANT: Make sure GPU is enabled:
   Runtime > Change runtime type > T4 GPU


## 2️⃣ Setup and Create Sample Documents

In [21]:
import os
from pathlib import Path

# Crear carpeta data si no existe
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# Documentos de ejemplo sobre ML/AI
SAMPLE_DOCS = {
    "machine_learning.txt": '''# Machine Learning - Fundamentos

Machine Learning (ML) es una rama de la inteligencia artificial que permite a los sistemas aprender y mejorar automáticamente a partir de la experiencia sin ser programados explícitamente.

## Tipos de Aprendizaje

### Aprendizaje Supervisado
El modelo aprende de datos etiquetados. Ejemplos:
- Clasificación: Spam detection, diagnóstico médico
- Regresión: Predicción de precios, pronóstico del tiempo

Algoritmos populares:
- Linear Regression
- Logistic Regression
- Random Forest
- Support Vector Machines (SVM)
- Gradient Boosting (XGBoost, LightGBM)

### Aprendizaje No Supervisado
El modelo encuentra patrones en datos sin etiquetar:
- Clustering: K-Means, DBSCAN, Hierarchical
- Reducción de dimensionalidad: PCA, t-SNE, UMAP
- Detección de anomalías: Isolation Forest

### Aprendizaje por Refuerzo
El agente aprende mediante interacción con el entorno:
- Q-Learning
- Deep Q-Networks (DQN)
- Policy Gradient Methods
- Actor-Critic (A2C, A3C, PPO)

## Proceso de ML

1. Recolección de datos: Obtener datos relevantes
2. Preprocesamiento: Limpieza, normalización, encoding
3. Feature Engineering: Crear características útiles
4. Entrenamiento: Ajustar el modelo a los datos
5. Evaluación: Métricas como accuracy, precision, recall, F1
6. Deployment: Poner el modelo en producción

## Métricas de Evaluación

### Clasificación
- Accuracy: Proporción de predicciones correctas
- Precision: TP / (TP + FP)
- Recall: TP / (TP + FN)
- F1-Score: Media armónica de precision y recall
- AUC-ROC: Área bajo la curva ROC

### Regresión
- MSE (Mean Squared Error)
- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)
- R2 (Coefficient of Determination)
''',

    "deep_learning.txt": '''# Deep Learning - Redes Neuronales Profundas

Deep Learning es un subconjunto de Machine Learning basado en redes neuronales artificiales con múltiples capas.

## Fundamentos

### Neurona Artificial (Perceptrón)
- Recibe entradas ponderadas
- Aplica una función de activación
- Produce una salida

### Funciones de Activación
- ReLU: max(0, x) - Más usada en capas ocultas
- Sigmoid: 1/(1+e^-x) - Salidas entre 0 y 1
- Tanh: Salidas entre -1 y 1
- Softmax: Para clasificación multiclase
- GELU: Usada en Transformers

## Arquitecturas Principales

### Redes Feedforward (MLP)
- Capas densamente conectadas
- Información fluye en una dirección
- Útil para datos tabulares

### Redes Convolucionales (CNN)
- Especializadas en procesamiento de imágenes
- Capas convolucionales detectan patrones locales
- Pooling reduce dimensionalidad
- Arquitecturas: LeNet, AlexNet, VGG, ResNet, EfficientNet

### Redes Recurrentes (RNN)
- Procesan secuencias de datos
- Mantienen estado interno (memoria)
- Variantes: LSTM, GRU
- Problema: Vanishing gradients en secuencias largas

### Transformers
- Arquitectura dominante actual
- Mecanismo de atención (self-attention)
- Procesamiento paralelo
- Base de GPT, BERT, T5, etc.

## Entrenamiento

### Backpropagation
- Calcula gradientes de la función de pérdida
- Propaga errores hacia atrás
- Actualiza pesos con optimizador

### Optimizadores
- SGD: Stochastic Gradient Descent
- Adam: Adaptive Moment Estimation (más popular)
- AdamW: Adam con weight decay
- RMSprop: Para RNNs

### Regularización
- Dropout: Desactiva neuronas aleatoriamente
- Batch Normalization: Normaliza activaciones
- Layer Normalization: Usada en Transformers
- Weight Decay (L2): Penaliza pesos grandes

## Frameworks
- PyTorch: Flexible, preferido en investigación
- TensorFlow/Keras: Producción, TensorFlow Serving
- JAX: Diferenciación automática, XLA
- Hugging Face: Modelos pre-entrenados de NLP
''',

    "transformers.txt": '''# Transformers - Arquitectura Revolucionaria

La arquitectura Transformer, introducida en "Attention is All You Need" (2017) por Vaswani et al., revolucionó el procesamiento de lenguaje natural.

## Componentes Clave

### Self-Attention (Atención)
Permite que cada token "atienda" a todos los demás tokens:

1. Query (Q): Lo que el token busca
2. Key (K): Lo que cada token ofrece
3. Value (V): La información real

Fórmula: Attention(Q,K,V) = softmax(QK^T / sqrt(d_k)) * V

### Multi-Head Attention
- Múltiples cabezas de atención en paralelo
- Cada cabeza aprende diferentes patrones
- Se concatenan y proyectan

### Positional Encoding
- Transformers no tienen noción de orden
- Se añade información posicional
- Encodings sinusoidales o aprendidos

### Feed-Forward Network (FFN)
- Dos capas lineales con activación
- Procesa cada posición independientemente
- Expande y contrae dimensionalidad

## Arquitecturas Derivadas

### Encoder-Only (BERT-style)
- Bidireccional, ve todo el contexto
- Ideal para clasificación, NER, embeddings
- Ejemplos: BERT, RoBERTa, ALBERT, DistilBERT

### Decoder-Only (GPT-style)
- Autoregresivo, genera token por token
- Ideal para generación de texto
- Ejemplos: GPT-2, GPT-3, GPT-4, LLaMA, Mistral

### Encoder-Decoder (T5-style)
- Encoder procesa entrada
- Decoder genera salida
- Ideal para traducción, summarization
- Ejemplos: T5, BART, mT5

## Large Language Models (LLMs)

### Características
- Billones de parámetros
- Entrenados en enormes corpus de texto
- Capacidades emergentes (few-shot, reasoning)

### Técnicas de Entrenamiento
- Pre-training: Modelado de lenguaje en corpus grande
- Fine-tuning: Ajuste para tarea específica
- RLHF: Reinforcement Learning from Human Feedback
- Instruction Tuning: Entrenamiento con instrucciones

### Optimizaciones
- LoRA: Low-Rank Adaptation (fine-tuning eficiente)
- QLoRA: LoRA con cuantización
- Flash Attention: Atención más rápida y eficiente
- Quantization: INT8, INT4 para menor memoria
''',

    "rag_systems.txt": '''# RAG - Retrieval-Augmented Generation

RAG combina recuperación de información con generación de lenguaje para crear sistemas que pueden responder preguntas basándose en documentos específicos.

## Por qué RAG?

### Limitaciones de LLMs puros
- Conocimiento desactualizado (fecha de corte)
- No acceso a información privada/propietaria
- Alucinaciones: inventan información falsa
- No pueden citar fuentes específicas

### Ventajas de RAG
- Información actualizada en tiempo real
- Acceso a documentos privados
- Respuestas fundamentadas en fuentes
- Reducción de alucinaciones
- Trazabilidad y citación

## Arquitectura RAG

### 1. Indexación (Offline)
1. Carga de documentos: PDFs, TXT, DOCX, HTML, etc.
2. Chunking: Dividir en fragmentos manejables
   - Por caracteres/tokens
   - Por oraciones/párrafos
   - Recursive character splitting
   - Semantic chunking
3. Embedding: Convertir texto a vectores
   - Sentence Transformers
   - Cohere Embeddings
4. Almacenamiento: Guardar en vector store
   - ChromaDB, FAISS, Pinecone, Weaviate, Milvus

### 2. Recuperación (Online)
1. Query embedding: Convertir pregunta a vector
2. Similarity search: Buscar chunks similares
   - Cosine similarity
   - Euclidean distance
   - Dot product
3. Reranking (opcional): Reordenar resultados

### 3. Generación
1. Prompt construction: Crear prompt con contexto
2. LLM call: Generar respuesta
3. Post-processing: Formatear, citar fuentes

## Técnicas Avanzadas

### Hybrid Search
- Combina búsqueda semántica + keyword (BM25)
- Mejor cobertura de consultas

### Multi-Query RAG
- Genera múltiples variantes de la pregunta
- Recupera documentos para cada variante
- Fusiona resultados

### Self-RAG
- El modelo decide cuándo recuperar
- Evalúa relevancia de documentos
- Auto-crítica de respuestas

### Agentic RAG
- Agentes que deciden qué herramientas usar
- Pueden iterar y refinar búsquedas
- Routing entre múltiples fuentes

## Vector Stores

### ChromaDB
- Open source, fácil de usar
- Perfecto para desarrollo y prototipos
- Persistencia local o en memoria

### FAISS (Facebook)
- Muy rápido, optimizado para GPU
- Ideal para grandes volúmenes

### Pinecone
- Servicio cloud gestionado
- Escalable, alta disponibilidad

### Weaviate
- Búsqueda híbrida nativa
- GraphQL API

## Métricas de Evaluación

### Retrieval
- Precision@K
- Recall@K
- MRR (Mean Reciprocal Rank)
- NDCG

### Generation
- Faithfulness: Respuesta fiel al contexto?
- Answer Relevance: Responde la pregunta?
- Context Relevance: Contexto relevante?
- Frameworks: RAGAS, TruLens
'''
}

# Guardar documentos de ejemplo
print("📁 Creando documentos de ejemplo...\n")
for filename, content in SAMPLE_DOCS.items():
    filepath = DATA_DIR / filename
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)
    print(f"✅ Creado: {filepath}")

print(f"\n📊 Total documentos: {len(list(DATA_DIR.glob('*')))}")

📁 Creando documentos de ejemplo...

✅ Creado: data/machine_learning.txt
✅ Creado: data/deep_learning.txt
✅ Creado: data/transformers.txt
✅ Creado: data/rag_systems.txt

📊 Total documentos: 4


## 3️⃣ Configuration - 100% FREE & UNLIMITED

This system uses a **local LLM** that runs on Colab's free GPU.

| Feature | Description |
|---------|-------------|
| **Model** | TinyLlama 1.1B Chat |
| **Cost** | 100% FREE |
| **Limits** | UNLIMITED |
| **API Key** | NOT REQUIRED |

In [22]:
import os
import torch

# ============================================
# 🎯 CONFIGURATION - 100% FREE & UNLIMITED
# ============================================
# Using a LOCAL model that runs on Colab's GPU
# No API key needed, no limits, completely free
# ============================================

# Check GPU availability
if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ GPU not detected. Go to: Runtime > Change runtime type > T4 GPU")

LLM_PROVIDER = "local"  # Local model, no API
print("\n✅ Configured for LOCAL model - 100% FREE & UNLIMITED")

✅ GPU available: Tesla T4
   Memory: 15.8 GB

✅ Configured for LOCAL model - 100% FREE & UNLIMITED


## 4️⃣ Load and Process Documents

In [23]:
from langchain_community.document_loaders import (
    DirectoryLoader,
    TextLoader,
    PyPDFLoader,
    Docx2txtLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_documents(data_path: str = "data"):
    """Load documents from multiple formats in a folder."""
    documents = []
    
    # Load .txt files
    txt_loader = DirectoryLoader(
        data_path, 
        glob="**/*.txt", 
        loader_cls=TextLoader,
        loader_kwargs={"encoding": "utf-8"}
    )
    try:
        txt_docs = txt_loader.load()
        documents.extend(txt_docs)
        print(f"📄 Loaded {len(txt_docs)} .txt files")
    except Exception as e:
        print(f"⚠️ Error loading .txt: {e}")
    
    # Load .pdf files
    pdf_loader = DirectoryLoader(
        data_path, 
        glob="**/*.pdf", 
        loader_cls=PyPDFLoader
    )
    try:
        pdf_docs = pdf_loader.load()
        documents.extend(pdf_docs)
        print(f"📄 Loaded {len(pdf_docs)} .pdf files")
    except Exception as e:
        print(f"⚠️ Error loading .pdf: {e}")
    
    # Load .docx files
    docx_loader = DirectoryLoader(
        data_path, 
        glob="**/*.docx", 
        loader_cls=Docx2txtLoader
    )
    try:
        docx_docs = docx_loader.load()
        documents.extend(docx_docs)
        print(f"📄 Loaded {len(docx_docs)} .docx files")
    except Exception as e:
        print(f"⚠️ Error loading .docx: {e}")
    
    return documents

# Load documents
print("📚 Loading documents...\n")
documents = load_documents("data")
print(f"\n✅ Total documents loaded: {len(documents)}")

📚 Loading documents...

📄 Loaded 4 .txt files
📄 Loaded 0 .pdf files
📄 Loaded 0 .docx files

✅ Total documents loaded: 4


In [24]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_documents(documents)

print(f"📊 Chunking Statistics:")
print(f"   • Original documents: {len(documents)}")
print(f"   • Generated chunks: {len(chunks)}")
if chunks:
    print(f"   • Average size: {sum(len(c.page_content) for c in chunks) // len(chunks)} characters")
    print(f"\n📝 Sample chunk:")
    print(f"{'='*50}")
    print(chunks[0].page_content[:300])
    print(f"{'='*50}")

📊 Chunking Statistics:
   • Original documents: 4
   • Generated chunks: 22
   • Average size: 372 characters

📝 Sample chunk:
# RAG - Retrieval-Augmented Generation

RAG combina recuperación de información con generación de lenguaje para crear sistemas que pueden responder preguntas basándose en documentos específicos.

## Por qué RAG?

### Limitaciones de LLMs puros
- Conocimiento desactualizado (fecha de corte)
- No acce


## 5️⃣ Create Vector Database with ChromaDB

In [25]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Configure embedding model (free, local)
print("🔄 Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Embedding model loaded")

# Directory to persist ChromaDB
CHROMA_PATH = "chroma_db"

# Create vector store with ChromaDB
print("\n🔄 Creating vector database...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=CHROMA_PATH,
    collection_name="rag_collection"
)

print(f"✅ ChromaDB created with {vectorstore._collection.count()} vectors")
print(f"📁 Persisted to: {CHROMA_PATH}/")

🔄 Loading embedding model...


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Embedding model loaded

🔄 Creating vector database...
✅ ChromaDB created with 44 vectors
📁 Persisted to: chroma_db/


## 6️⃣ Test Semantic Search

In [26]:
def search_similar(query: str, k: int = 3):
    """Search for the k most similar documents to the query."""
    results = vectorstore.similarity_search_with_score(query, k=k)
    
    print(f"🔍 Query: '{query}'\n")
    print(f"📚 Top {k} results:\n")
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"{'='*60}")
        print(f"📄 Result #{i} (Score: {score:.4f})")
        print(f"📁 Source: {doc.metadata.get('source', 'N/A')}")
        print(f"{'='*60}")
        print(doc.page_content[:400])
        print()
    
    return results

# Test search
results = search_similar("What is attention in transformers?")

🔍 Query: 'What is attention in transformers?'

📚 Top 3 results:

📄 Result #1 (Score: 0.9635)
📁 Source: data/transformers.txt
# Transformers - Arquitectura Revolucionaria

La arquitectura Transformer, introducida en "Attention is All You Need" (2017) por Vaswani et al., revolucionó el procesamiento de lenguaje natural.

## Componentes Clave

### Self-Attention (Atención)
Permite que cada token "atienda" a todos los demás tokens:

1. Query (Q): Lo que el token busca
2. Key (K): Lo que cada token ofrece
3. Value (V): La in

📄 Result #2 (Score: 0.9635)
📁 Source: data/transformers.txt
# Transformers - Arquitectura Revolucionaria

La arquitectura Transformer, introducida en "Attention is All You Need" (2017) por Vaswani et al., revolucionó el procesamiento de lenguaje natural.

## Componentes Clave

### Self-Attention (Atención)
Permite que cada token "atienda" a todos los demás tokens:

1. Query (Q): Lo que el token busca
2. Key (K): Lo que cada token ofrece
3. Value (V): La in

📄 Result #3

In [27]:
# More search examples
results = search_similar("How does RAG work and what are its advantages?")

🔍 Query: 'How does RAG work and what are its advantages?'

📚 Top 3 results:

📄 Result #1 (Score: 1.1671)
📁 Source: data/rag_systems.txt
### Ventajas de RAG
- Información actualizada en tiempo real
- Acceso a documentos privados
- Respuestas fundamentadas en fuentes
- Reducción de alucinaciones
- Trazabilidad y citación

## Arquitectura RAG

📄 Result #2 (Score: 1.1671)
📁 Source: data/rag_systems.txt
### Ventajas de RAG
- Información actualizada en tiempo real
- Acceso a documentos privados
- Respuestas fundamentadas en fuentes
- Reducción de alucinaciones
- Trazabilidad y citación

## Arquitectura RAG

📄 Result #3 (Score: 1.3461)
📁 Source: data/rag_systems.txt
# RAG - Retrieval-Augmented Generation

RAG combina recuperación de información con generación de lenguaje para crear sistemas que pueden responder preguntas basándose en documentos específicos.

## Por qué RAG?

### Limitaciones de LLMs puros
- Conocimiento desactualizado (fecha de corte)
- No acceso a información privada/propiet

## 7️⃣ Configure LLM and Create RAG Chain

In [28]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# ============================================
# Load LOCAL model - 100% FREE & UNLIMITED
# ============================================
print("🔄 Loading model (may take 1-2 minutes the first time)...")

# Using TinyLlama - lightweight but effective, works well on Colab
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Create generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15,
    do_sample=True,
)

# Wrapper for LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print(f"✅ Model loaded: {model_id}")
print("🎉 100% FREE & UNLIMITED - No API key, no limits")

🔄 Loading model (may take 1-2 minutes the first time)...


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

✅ Model loaded: TinyLlama/TinyLlama-1.1B-Chat-v1.0
🎉 100% FREE & UNLIMITED - No API key, no limits


In [29]:
# Create RAG prompt template (optimized for local model)
RAG_TEMPLATE = """<|system|>
You are an expert assistant that answers questions based ONLY on the provided context.
Answer concisely and accurately. If the information is not in the context, say "I don't have enough information to answer".
</s>
<|user|>
CONTEXT:
{context}

QUESTION: {question}
</s>
<|assistant|>
"""

RAG_PROMPT = PromptTemplate(
    template=RAG_TEMPLATE,
    input_variables=["context", "question"]
)

def format_docs(docs):
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"[{source}]: {doc.page_content}")
    return "\n\n".join(formatted)

# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Reduced to 3 for better performance
)

# Create RAG chain with LCEL
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | RAG_PROMPT
    | llm
    | StrOutputParser()
)

print("✅ RAG chain configured - 100% LOCAL and FREE")

✅ RAG chain configured - 100% LOCAL and FREE


## 8️⃣ Ask Questions! 🎉

In [30]:
def ask(question: str, show_sources: bool = True):
    """Function to ask questions to the RAG system."""
    print(f"❓ Question: {question}\n")
    print("🔄 Processing...\n")
    
    response = rag_chain.invoke(question)
    
    print("💬 Answer:")
    print("="*60)
    print(response)
    print("="*60)
    
    if show_sources:
        docs = retriever.invoke(question)
        print("\n📚 Sources used:")
        sources = set(doc.metadata.get('source', 'N/A') for doc in docs)
        for source in sources:
            print(f"   • {source}")
    
    return response

In [31]:
# Example 1: Question about Transformers
ask("What is the attention mechanism in Transformers and how does it work?")

Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


❓ Question: What is the attention mechanism in Transformers and how does it work?

🔄 Processing...

💬 Answer:
<|system|>
You are an expert assistant that answers questions based ONLY on the provided context.
Answer concisely and accurately. If the information is not in the context, say "I don't have enough information to answer".
</s>
<|user|>
CONTEXT:
[data/transformers.txt]: # Transformers - Arquitectura Revolucionaria

La arquitectura Transformer, introducida en "Attention is All You Need" (2017) por Vaswani et al., revolucionó el procesamiento de lenguaje natural.

## Componentes Clave

### Self-Attention (Atención)
Permite que cada token "atienda" a todos los demás tokens:

1. Query (Q): Lo que el token busca
2. Key (K): Lo que cada token ofrece
3. Value (V): La información real

Fórmula: Attention(Q,K,V) = softmax(QK^T / sqrt(d_k)) * V

[data/transformers.txt]: # Transformers - Arquitectura Revolucionaria

La arquitectura Transformer, introducida en "Attention is All You Need" (2

'<|system|>\nYou are an expert assistant that answers questions based ONLY on the provided context.\nAnswer concisely and accurately. If the information is not in the context, say "I don\'t have enough information to answer".\n</s>\n<|user|>\nCONTEXT:\n[data/transformers.txt]: # Transformers - Arquitectura Revolucionaria\n\nLa arquitectura Transformer, introducida en "Attention is All You Need" (2017) por Vaswani et al., revolucionó el procesamiento de lenguaje natural.\n\n## Componentes Clave\n\n### Self-Attention (Atención)\nPermite que cada token "atienda" a todos los demás tokens:\n\n1. Query (Q): Lo que el token busca\n2. Key (K): Lo que cada token ofrece\n3. Value (V): La información real\n\nFórmula: Attention(Q,K,V) = softmax(QK^T / sqrt(d_k)) * V\n\n[data/transformers.txt]: # Transformers - Arquitectura Revolucionaria\n\nLa arquitectura Transformer, introducida en "Attention is All You Need" (2017) por Vaswani et al., revolucionó el procesamiento de lenguaje natural.\n\n## Comp

In [32]:
# Example 2: Question about RAG
ask("What are the advantages of using RAG instead of a pure LLM?")

Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


❓ Question: What are the advantages of using RAG instead of a pure LLM?

🔄 Processing...

💬 Answer:
<|system|>
You are an expert assistant that answers questions based ONLY on the provided context.
Answer concisely and accurately. If the information is not in the context, say "I don't have enough information to answer".
</s>
<|user|>
CONTEXT:
[data/rag_systems.txt]: # RAG - Retrieval-Augmented Generation

RAG combina recuperación de información con generación de lenguaje para crear sistemas que pueden responder preguntas basándose en documentos específicos.

## Por qué RAG?

### Limitaciones de LLMs puros
- Conocimiento desactualizado (fecha de corte)
- No acceso a información privada/propietaria
- Alucinaciones: inventan información falsa
- No pueden citar fuentes específicas

[data/rag_systems.txt]: # RAG - Retrieval-Augmented Generation

RAG combina recuperación de información con generación de lenguaje para crear sistemas que pueden responder preguntas basándose en documentos espec

'<|system|>\nYou are an expert assistant that answers questions based ONLY on the provided context.\nAnswer concisely and accurately. If the information is not in the context, say "I don\'t have enough information to answer".\n</s>\n<|user|>\nCONTEXT:\n[data/rag_systems.txt]: # RAG - Retrieval-Augmented Generation\n\nRAG combina recuperación de información con generación de lenguaje para crear sistemas que pueden responder preguntas basándose en documentos específicos.\n\n## Por qué RAG?\n\n### Limitaciones de LLMs puros\n- Conocimiento desactualizado (fecha de corte)\n- No acceso a información privada/propietaria\n- Alucinaciones: inventan información falsa\n- No pueden citar fuentes específicas\n\n[data/rag_systems.txt]: # RAG - Retrieval-Augmented Generation\n\nRAG combina recuperación de información con generación de lenguaje para crear sistemas que pueden responder preguntas basándose en documentos específicos.\n\n## Por qué RAG?\n\n### Limitaciones de LLMs puros\n- Conocimiento d

In [33]:
# Example 3: Question about Deep Learning
ask("What optimizers are used to train neural networks?")

Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


❓ Question: What optimizers are used to train neural networks?

🔄 Processing...

💬 Answer:
<|system|>
You are an expert assistant that answers questions based ONLY on the provided context.
Answer concisely and accurately. If the information is not in the context, say "I don't have enough information to answer".
</s>
<|user|>
CONTEXT:
[data/deep_learning.txt]: ### Regularización
- Dropout: Desactiva neuronas aleatoriamente
- Batch Normalization: Normaliza activaciones
- Layer Normalization: Usada en Transformers
- Weight Decay (L2): Penaliza pesos grandes

## Frameworks
- PyTorch: Flexible, preferido en investigación
- TensorFlow/Keras: Producción, TensorFlow Serving
- JAX: Diferenciación automática, XLA
- Hugging Face: Modelos pre-entrenados de NLP

[data/deep_learning.txt]: ### Regularización
- Dropout: Desactiva neuronas aleatoriamente
- Batch Normalization: Normaliza activaciones
- Layer Normalization: Usada en Transformers
- Weight Decay (L2): Penaliza pesos grandes

## Frameworks


'<|system|>\nYou are an expert assistant that answers questions based ONLY on the provided context.\nAnswer concisely and accurately. If the information is not in the context, say "I don\'t have enough information to answer".\n</s>\n<|user|>\nCONTEXT:\n[data/deep_learning.txt]: ### Regularización\n- Dropout: Desactiva neuronas aleatoriamente\n- Batch Normalization: Normaliza activaciones\n- Layer Normalization: Usada en Transformers\n- Weight Decay (L2): Penaliza pesos grandes\n\n## Frameworks\n- PyTorch: Flexible, preferido en investigación\n- TensorFlow/Keras: Producción, TensorFlow Serving\n- JAX: Diferenciación automática, XLA\n- Hugging Face: Modelos pre-entrenados de NLP\n\n[data/deep_learning.txt]: ### Regularización\n- Dropout: Desactiva neuronas aleatoriamente\n- Batch Normalization: Normaliza activaciones\n- Layer Normalization: Usada en Transformers\n- Weight Decay (L2): Penaliza pesos grandes\n\n## Frameworks\n- PyTorch: Flexible, preferido en investigación\n- TensorFlow/Ke

In [34]:
# Example 4: Question about Machine Learning
ask("Explain the differences between supervised and unsupervised learning")

Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


❓ Question: Explain the differences between supervised and unsupervised learning

🔄 Processing...

💬 Answer:
<|system|>
You are an expert assistant that answers questions based ONLY on the provided context.
Answer concisely and accurately. If the information is not in the context, say "I don't have enough information to answer".
</s>
<|user|>
CONTEXT:
[data/machine_learning.txt]: # Machine Learning - Fundamentos

Machine Learning (ML) es una rama de la inteligencia artificial que permite a los sistemas aprender y mejorar automáticamente a partir de la experiencia sin ser programados explícitamente.

## Tipos de Aprendizaje

### Aprendizaje Supervisado
El modelo aprende de datos etiquetados. Ejemplos:
- Clasificación: Spam detection, diagnóstico médico
- Regresión: Predicción de precios, pronóstico del tiempo

[data/machine_learning.txt]: # Machine Learning - Fundamentos

Machine Learning (ML) es una rama de la inteligencia artificial que permite a los sistemas aprender y mejorar automá

'<|system|>\nYou are an expert assistant that answers questions based ONLY on the provided context.\nAnswer concisely and accurately. If the information is not in the context, say "I don\'t have enough information to answer".\n</s>\n<|user|>\nCONTEXT:\n[data/machine_learning.txt]: # Machine Learning - Fundamentos\n\nMachine Learning (ML) es una rama de la inteligencia artificial que permite a los sistemas aprender y mejorar automáticamente a partir de la experiencia sin ser programados explícitamente.\n\n## Tipos de Aprendizaje\n\n### Aprendizaje Supervisado\nEl modelo aprende de datos etiquetados. Ejemplos:\n- Clasificación: Spam detection, diagnóstico médico\n- Regresión: Predicción de precios, pronóstico del tiempo\n\n[data/machine_learning.txt]: # Machine Learning - Fundamentos\n\nMachine Learning (ML) es una rama de la inteligencia artificial que permite a los sistemas aprender y mejorar automáticamente a partir de la experiencia sin ser programados explícitamente.\n\n## Tipos de 

## 9️⃣ Interactive Chat

In [35]:
# Interactive chat
print("🤖 Interactive RAG System")
print("="*40)
print("Type your questions (type 'exit' to quit)\n")

while True:
    try:
        question = input("\n❓ Your question: ")
        if question.lower() in ['exit', 'quit', 'q', 'salir']:
            print("\n👋 Goodbye!")
            break
        if question.strip():
            ask(question)
    except KeyboardInterrupt:
        print("\n\n👋 Goodbye!")
        break

🤖 Interactive RAG System
Type your questions (type 'exit' to quit)



Both `max_new_tokens` (=512) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


❓ Question: what is a neuron ?

🔄 Processing...

💬 Answer:
<|system|>
You are an expert assistant that answers questions based ONLY on the provided context.
Answer concisely and accurately. If the information is not in the context, say "I don't have enough information to answer".
</s>
<|user|>
CONTEXT:
[data/deep_learning.txt]: # Deep Learning - Redes Neuronales Profundas

Deep Learning es un subconjunto de Machine Learning basado en redes neuronales artificiales con múltiples capas.

## Fundamentos

### Neurona Artificial (Perceptrón)
- Recibe entradas ponderadas
- Aplica una función de activación
- Produce una salida

[data/deep_learning.txt]: # Deep Learning - Redes Neuronales Profundas

Deep Learning es un subconjunto de Machine Learning basado en redes neuronales artificiales con múltiples capas.

## Fundamentos

### Neurona Artificial (Perceptrón)
- Recibe entradas ponderadas
- Aplica una función de activación
- Produce una salida

[data/deep_learning.txt]: ### Redes Convoluciona

---

## 🎯 Conclusion

### What We Built
In this notebook, we implemented a complete **RAG (Retrieval-Augmented Generation)** system from scratch using:

| Component | Technology | Cost |
|-----------|------------|------|
| **Vector Database** | ChromaDB | FREE |
| **Embeddings** | Sentence Transformers (all-MiniLM-L6-v2) | FREE |
| **LLM** | TinyLlama 1.1B (Local) | FREE & UNLIMITED |
| **Orchestration** | LangChain | FREE |

### Key Learnings

1. **RAG Architecture**: We learned how to combine retrieval and generation to create more accurate and grounded AI responses.

2. **Vector Databases**: ChromaDB provides efficient similarity search for finding relevant document chunks.

3. **Embeddings**: Sentence Transformers convert text to vectors that capture semantic meaning.

4. **Local LLMs**: Running models locally on Colab's GPU eliminates API costs and rate limits.

5. **LangChain**: Simplifies building complex AI pipelines with its modular components.

### Advantages of This Approach

- ✅ **100% Free**: No API keys, no subscriptions, no hidden costs
- ✅ **Unlimited Usage**: No rate limits or quotas
- ✅ **Privacy**: Your data never leaves Colab's environment
- ✅ **Reproducible**: Everything runs in a single notebook
- ✅ **Extensible**: Easy to add new documents or swap components

### Next Steps

1. **Add your own documents**: Upload PDFs, DOCX, or TXT files to the `data/` folder
2. **Try larger models**: If you have more GPU memory, try Mistral 7B or LLaMA 2
3. **Implement hybrid search**: Combine semantic search with keyword search (BM25)
4. **Add reranking**: Use a cross-encoder to rerank retrieved documents
5. **Build a UI**: Create a Gradio or Streamlit interface for better UX

### Resources

- 📚 [LangChain Documentation](https://python.langchain.com/)
- 📚 [ChromaDB Documentation](https://docs.trychroma.com/)
- 📚 [Hugging Face Transformers](https://huggingface.co/docs/transformers/)
- 📚 [RAG Paper (Lewis et al., 2020)](https://arxiv.org/abs/2005.11401)

---

**🎉 Congratulations! You've built a complete RAG system that runs 100% free and unlimited on Google Colab!**